<a href="https://colab.research.google.com/github/jrg70/Co-crystallizationML/blob/main/Predicting_Mechanochemical_Co_crystallization_via_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The code below will:

- Clean descriptor data
- Create the final experimental matrix
- Normalization and apply dimension reduction (PCA)
- Set up machine learning data frame and randomize
- Set up model and execute cross-validation
- Hyperparameter tuning

*All for the first aim (neat grinding experiments)*

**Let's start**

1. Load external data

1.1. Raw molecular descriptor data of all components

In [ ]:
import pandas as pd

descriptors = pd.read_csv('Chem.csv', index_col=0)
print(descriptors.info())

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 2-Pyrrolidinone to 3-Hydroxybenzamide
Columns: 1824 entries, ABC to mZagreb2
dtypes: float64(1458), int64(366)
memory usage: 1.8+ MB
None


1.2. Cocrystal combinations and characterisation results

In [ ]:
results = pd.read_csv('Results.csv')
print(results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Component1  1000 non-null   object
 1   Component2  1000 non-null   object
 2   Outcome     1000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 23.6+ KB
None


2. Descriptor data cleaning

2.1. Create final data frame and drop columns with missing values

In [ ]:
descriptor_set = descriptors.dropna(axis=1)

2.2. Drop columns with only a single unique value over all rows

In [ ]:
non_unique = descriptor_set.apply(pd.Series.nunique)
cols_to_drop = non_unique[non_unique == 1].index
descriptor_set = descriptor_set.drop(cols_to_drop, axis=1)
print(descriptor_set.info())

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 2-Pyrrolidinone to 3-Hydroxybenzamide
Columns: 1003 entries, ABC to mZagreb2
dtypes: float64(855), int64(148)
memory usage: 996.2+ KB
None


The new number of descriptors is reduced from 1824 to 1041.

2.3. Exporting the actual descriptor_set (virtual library in csv)
#descriptor_set.to_csv('descriptor_set.csv')

3. Create final experimental matrix containing descriptors and results

3.1. Column descriptors CC_5 for final matrix

In [ ]:
CC_1 = descriptor_set.columns + '_1'
CC_2 = descriptor_set.columns + '_2'
CC_3 = ['Result']
CC_4 = CC_1.union(CC_2, sort=False)
CC_5 = CC_4.union(CC_3, sort=False)

3.2. Initialize empty array and set random seed so that shuffle is the same very time

In [ ]:
import numpy as np

CC_e = []
np.random.seed(1)

3.3. Fill empty array with Component 1, Component 2 descriptors and the result (per row/experiment)

In [ ]:
from sklearn.utils import shuffle

for i in range(len(results)):
    A = np.array(descriptor_set.loc[results.iloc[i][0]])
    B = np.array(descriptor_set.loc[results.iloc[i][1]])
    RS = [A, B]
    C = np.concatenate(shuffle(RS), axis=0)
    D = [results.iloc[i][-1]]
    E = np.concatenate((C, D), axis=0)
    CC_e.append(E)

3.4. Transform array in data frame to obtain final matrix

In [ ]:
CC = pd.DataFrame(CC_e, columns=CC_5)
print(CC.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 2007 entries, ABC_1 to Result
dtypes: float64(2007)
memory usage: 15.3 MB
None


4. Normalize data and apply PCA

4.1. Normalization of the descriptors

In [ ]:
from sklearn.preprocessing import StandardScaler

x = CC.loc[:, CC_4].values
x = StandardScaler().fit_transform(x)
sep = int((x.shape[1]*0.5))
x1 = (x[:, :sep])
x2 = (x[:, sep:])

4.2. Normalized data has a mean of 0 and standard deviation of 1 (Proof)

In [ ]:
print('The mean of the normalized data is {} and the standard deviation is {}.'.format(np.mean(x), np.std(x)))


The mean of the normalized data is 2.2811042962587465e-18 and the standard deviation is 0.9849313330418408.


4.3. Apply PCA

4.3.1. For component 1

In [ ]:
from sklearn.decomposition import PCA

pca_components = 25
CC_pca1 = PCA(n_components=pca_components)
CC_princomp1 = CC_pca1.fit_transform(x1)
CC_principal_df1 = pd.DataFrame(data=CC_princomp1, columns=['PC_1_' + str(i + 1) for i in range(pca_components)])

4.3.2. For component 2

In [ ]:
CC_pca2 = PCA(n_components=pca_components)
CC_princomp2 = CC_pca2.fit_transform(x2)
CC_principal_df2 = pd.DataFrame(data=CC_princomp2, columns=['PC_2_' + str(i + 1) for i in range(pca_components)])

4.3.3. Print dimension reduction results

In [ ]:
Lost1 = (100*(1-sum(CC_pca1.explained_variance_ratio_)))
nb_des1 = x1.shape[1]
nb_com1 = len(CC_pca1.explained_variance_ratio_)
print('{}% of the information is lost by reducing {} features to {}.'.format(Lost1, nb_des1, nb_com1))

Lost2 = (100*(1-sum(CC_pca2.explained_variance_ratio_)))
nb_des2 = x2.shape[1]
nb_com2 = len(CC_pca2.explained_variance_ratio_)
print('{}% of the information is lost by reducing {} features to {}.'.format(Lost2, nb_des2, nb_com2))

1.888515326421647% of the information is lost by reducing 1003 features to 25.
1.611298821390983% of the information is lost by reducing 1003 features to 25.


5. Set up machine learning data frame and randomize

5.1. Create the final data frame

In [ ]:
CC_principal_together = pd.concat([CC_principal_df1, CC_principal_df2], axis=1)
CC_df = pd.concat([CC_principal_together, CC.iloc[:, -1]], axis=1)

5.2. Start randomizing by setting random seed and put rows into random order: permute index randomly and reindex data frame with that result

In [ ]:
np.random.seed(2)
CC_df = CC_df.reindex(np.random.permutation(CC_df.index))

6. Model evaluation

6.1. Set up classifiers and choose one from the list

In [ ]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
! pip install xgboost
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier()
# clf = ExtraTreesClassifier()
# clf = AdaBoostClassifier()
# clf = GradientBoostingClassifier()
# clf = svm.SVC()
# clf = xgb.XGBClassifier()

6.2. Cross-validation (five-fold) results in mean accuracy and standard deviation

In [ ]:
scores = cross_val_score(clf, CC_df.iloc[:, :-1], CC_df.iloc[:, -1], cv=5)
print("Accuracy: {} (+/- {})".format(scores.mean(), scores.std() * 2))
print(scores)

Accuracy: 0.708 (+/- 0.0427083130081252)
[0.735 0.715 0.67  0.705 0.715]


7. Hyperparameter tuning

7.1. Prepare everything for tuning

In [ ]:
! pip install hyperopt
import hyperopt as hp
from hyperopt import Trials, fmin, STATUS_OK

list_mean = list()
list_std = list()

7.2. Defining the objective function

In [ ]:
def objective(params, n_folds=5):
    d_train = xgb.DMatrix(CC_df.iloc[:, :-1], CC_df.iloc[:, -1])

    cv_results = xgb.cv(params, d_train, nfold=n_folds, num_boost_round=500,
                        early_stopping_rounds=25, metrics='auc', seed=0)

    auc = max(cv_results['test-auc-mean'])
    list_mean.append(auc)
    std = cv_results.iloc[-1][-1]
    list_std.append(std)
    return 1-auc

7.3. Search space 
- Verbosity: do not print warning for not included estimators
- max_depth: maximum depth allowed for every tree growth
- subsample: maximum allowed rows for every tree
- n_estimators: number of trees you want to build
- colsample_bytree: maximum allowed features for every tree
- min_child-weight: minimum number of instances required in each node
- reg_alpha: L1 regularisation term on weights
- reg_lambda: L2 regularisation term on weights

In [ ]:
xgb_space = {
    'verbosity': 0,
    'max_depth': hp.hp.choice('max_depth', np.arange(1, 21, 1, dtype=int)),
    'subsample': hp.hp.quniform('subsample', 0.5, 1.0, 0.05),
    'n_estimators': hp.hp.quniform('n_estimators', 2, 201, 1),
    'colsample_bytree': hp.hp.quniform('colsample_bytree', 0.5, 1.0, 0.05),
    'min_child_weight': hp.hp.uniform('min_child_weight', 1, 50),
    'reg_alpha': hp.hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.hp.uniform('reg_lambda', 0.0, 1.0)
    }

7.4. Storing the results of every iteration and optimize

In [ ]:
trials = Trials()
MAX_EVALS = 200
best = fmin(fn=objective, space=xgb_space, algo=hp.tpe.suggest,
            max_evals=MAX_EVALS, trials=trials)


100%|██████████| 200/200 [01:29<00:00,  2.23trial/s, best loss: 0.2009476]        


7.5. Print best parameter set, mean accuracy and standard deviation

In [ ]:
print(best)

print("Accuracy: {} (+/- {})".format(max(list_mean), list_std[list_mean.index(max(list_mean))]))

{'colsample_bytree': 0.7000000000000001, 'max_depth': 11, 'min_child_weight': 45.06100407599634, 'n_estimators': 110.0, 'reg_alpha': 0.676410817353635, 'reg_lambda': 0.2314032507189823, 'subsample': 1.0}
Accuracy: 0.7990524 (+/- 0.017233905820794076)


8. Testing a paracetamol set, the original data frame has to be changed to a Paratest set and then the follwoing be executed for accuracy, precision scroes and confusion matrix

In [ ]:
#clf = xgb.XGBClassifier(xgb_space=best)
#clf.fit(Paratest_df.iloc[36:, 0:-1], Paratest_df.iloc[36:, -1])
#predictions = clf.predict(Paratest_df.iloc[0:35, 0:-1])
#auc1 = roc_auc_score(Paratest_df.iloc[0:35, -1], predictions)
#print(auc1)

#print('Accuracy Score: ' + str(accuracy_score(Paratest_df.iloc[0:35, -1],predictions)))
#print('Precision Score: ' + str(precision_score(Paratest_df.iloc[0:35, -1],predictions)))
#print('Confusion Matrix: \n' + str(confusion_matrix(Paratest_df.iloc[0:35, -1],predictions))))